In [1]:
import tifffile
import numpy as np
from tqdm import tqdm
from pathlib import Path
import os
import pandas as pd
import sys
sys.path.append(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\10_code\UTvsXCT-preprocessing')
from dbtools import dbtools as db
from dbtools import load
from preprocess_tools import io, aligner, reslicer, signal, register

# Database conection

In [2]:
try:
    conn = db.connect()
    print("Connected to the database")
except Exception as error:
    print(error)

Connected to the database


# Measurement type id

Select the measurement type of the monoelement and the xct

In [3]:
measurementtypes_table = db.get_data_metadata('measurementtypes')

measurementtypes_table

,id_measurementtype,name_measurementtype,technique_measurementtype,equipment_measurementtype,transducer_name_measurementtype,transducer_nominal_freq_measurementtype,transducer_diameter_measurementtype,transducer_focal_length_measurementtype,z_resolution_measurementtype,x_resolution_measurementtype,...,current_measurementtype,voxel_size_measurementtype,images_taken_per_projection_measurementtype,exposure_time_per_image_taken_measurementtype,binning_measurementtype,projections_measurementtype,target_measurementtype,filter_measurementtype,Detector_measurementtype,DAC_measurementtype
0,5,Monoelement Ultrasound Pulse Echo 1,Ultrasound Monoelement Pulse Echo nominal,TRITON 1000P nominal,Technisonic ISL-0503-HR nominal,5 MHz,9.525 mm,76.2 mm,0.02232141 mm,1 mm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,XCT 1,X-Ray Computed Tomography nominal,ZEISS Xradia Versa 620 nominal,NaN,NaN,NaN,NaN,NaN,NaN,...,90 uA,0.025 mm,5 cardinal,0.2 s,1 cardinal,3738 cardinal,wolframio nominal,LE1 nominal,Flat Panel nominal,NaN
2,7,Monoelement Ultrasound Pulse Echo Airbus DAC,Ultrasound Monoelement Pulse Echo nominal,TRITON 1000P nominal,Technisonic ISL-0503-HR nominal,5 MHz,9.525 mm,76.2 mm,0.02232141 mm,1 mm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[10.1,4.33],[12.1,6.18],[19.7,10.38],[26.7,15..."
3,8,Monoelement Ultrasound Pulse Echo Low Resolution,Ultrasound Monoelement Pulse Echo nominal,TRITON 1000P nominal,Technisonic ISL-0503-HR nominal,5 MHz,9.525 mm,76.2 mm,0.05940588 mm,1 mm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
reference_measurementtype_id = 7
registered_measurementtype_id = 6

In [5]:
#get the resolutions of each type
reference_resolution = 1
registered_resolution = measurementtypes_table[measurementtypes_table['id_measurementtype'] == registered_measurementtype_id]['voxel_size_measurementtype'].values[0]

registered_resolution = float(registered_resolution.split(' ')[0])

print(f"Reference resolution: {reference_resolution}"
      f"\nRegistered resolution: {registered_resolution}")

Reference resolution: 1
Registered resolution: 0.025


# Measurement filtering

We have to filter the measurements to select which of them to register.

## Select samples

We select the panel

In [6]:
panels_table = db.get_data_metadata('panels')

panels_table

,id_panel,name_panel,material_id_panel,description_panel,fabrication_id_panel,height_panel,width_panel,thickness_panel,edges_cutted_panel,layer_layout_panel,geometry_panel
0,3,Juan_Ignacio_placeholder,15,This is a placeholder because we dont know fro...,2,1.0 mm,1.0 mm,1.0 mm,True bool,"[-45, 90, 45, 0, -45, 90, 45, 0, 0, 45, 90, -4...",Paralell faces nominal
1,4,Pegaso Piramide 7922843,16,The panel features multiple height levels rang...,3,945.0 mm,1150.0 mm,5.0 mm,True bool,"[45, -45, 90, 0, 45, -45, 0, 90, -45, 45] list",pyramid nominal


In [7]:
panel_id = 4

Get all the samples from the selected panel.

Get only the ones with keyholes.

Get only the id and name of the samples.

In [8]:
samples_table = db.get_data_metadata('samples')

samples_table = samples_table[samples_table['panel_id_sample'] == panel_id]

samples_table = samples_table[samples_table['keyhole_sample'] == 'True bool']

samples_table = samples_table[['id_sample', 'name_sample']]

samples_table

,id_sample,name_sample
8,17,1.7
9,18,1.8
10,19,1.9
11,20,1.10
12,21,1.11
13,22,1.12
14,23,1.13
15,24,1.14
16,25,1.15
17,26,1.16


## select measurements

We have to filter the measurements

### Monoelement filtering

In [9]:
measurements_table_monoelement = db.relation_metadata('measurements', 'samples','sample_measurements')

measurements_table_monoelement = measurements_table_monoelement[measurements_table_monoelement['measurementtype_id_measurement'] == reference_measurementtype_id]

#drop NA columns
measurements_table_monoelement = measurements_table_monoelement.dropna(axis=1, how='all')

#get the measurements from the samples
measurements_table_monoelement = measurements_table_monoelement[measurements_table_monoelement['id_sample'].isin(samples_table['id_sample'])]

#group by file_path_measurement
def agg_func(x):
    if len(x.unique()) > 1:
        return list(x.unique())
    else:
        return x.iloc[0]

measurements_table_monoelement = measurements_table_monoelement.groupby(
    ['file_path_measurement'], dropna=False
).agg(agg_func).reset_index()

#get the measurements from the samples
measurements_table_monoelement = measurements_table_monoelement[measurements_table_monoelement['id_sample'].isin(samples_table['id_sample'])]

measurements_table_monoelement = measurements_table_monoelement[['file_path_measurement','id_measurement','id_sample','name_sample']]

measurements_table_monoelement

,file_path_measurement,id_measurement,id_sample,name_sample
2,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,138,32,1.28
4,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,140,34,1.34
5,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,139,37,1.35
7,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,178,20,1.10
8,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,179,21,1.11
9,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,180,22,1.12
10,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,181,23,1.13
11,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,182,24,1.14
12,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,183,25,1.15
13,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,184,26,1.16


### xct filtering

In [10]:
measurements_table_xct = db.relation_metadata('measurements', 'samples','sample_measurements')

measurements_table_xct = measurements_table_xct[measurements_table_xct['measurementtype_id_measurement'] == registered_measurementtype_id]

#drop NA columns
measurements_table_xct = measurements_table_xct.dropna(axis=1, how='all')

#get the measurements from the samples
measurements_table_xct = measurements_table_xct[measurements_table_xct['id_sample'].isin(samples_table['id_sample'])]

#group by file_path_measurement
def agg_func(x):
    if len(x.unique()) > 1:
        return list(x.unique())
    else:
        return x.iloc[0]

measurements_table_xct = measurements_table_xct.groupby(
    ['file_path_measurement'], dropna=False
).agg(agg_func).reset_index()

#get the measurements from the samples
measurements_table_xct = measurements_table_xct[measurements_table_xct['id_sample'].isin(samples_table['id_sample'])]

#get the equalized measurements
measurements_table_xct = measurements_table_xct[measurements_table_xct['equalized_measurement'] == 'True bool']

#get the aligned measurements
measurements_table_xct = measurements_table_xct[measurements_table_xct['aligned_measurement'] == 'True bool']


# measurements_table_xct = measurements_table_xct[['file_path_measurement','id_measurement','id_sample','name_sample']]

measurements_table_xct[['file_path_measurement','id_measurement','id_sample','name_sample']]

,file_path_measurement,id_measurement,id_sample,name_sample
13,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,146,20,1.10
15,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,147,21,1.11
17,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,148,22,1.12
19,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,149,23,1.13
21,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,162,24,1.14
23,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,163,25,1.15
25,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,164,26,1.16
27,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,165,27,1.17
30,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,229,28,1.21
33,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,230,29,1.22


## Get existing registrations

In [11]:
registration_type = 'Alberto 2024 registration method, UTvsXCTPreprocessing toolkit 0.1.14 , file register.py function register_ut_xct_monoelement. Extract the centers of the holes from UT and XCT, and register them using a rigid body transformation text'

registrations_table = db.get_data_metadata('registrations')

try:
    registrations_table = registrations_table[registrations_table['type_registration'] == registration_type]

    registrations_table

    available_registrations = True

except Exception as e:
    print(f"No registrations available for type: {registration_type}")
    available_registrations = False


# Registering

In [ ]:
for i in range(len(samples_table)):
        
    try:

        print(f"Processing sample {i+1}/{len(samples_table)}: {samples_table['name_sample'].iloc[i]}")

        #get the measurements for the sample
        measurements_table_reference_sample = measurements_table_monoelement[measurements_table_monoelement['id_sample'] == samples_table['id_sample'].iloc[i]]

        if measurements_table_reference_sample.empty:
            print(f"No reference measurements found for sample {samples_table['name_sample'].iloc[i]}")
            continue

        #get the measurements for the sample
        measurements_table_registered_sample = measurements_table_xct[measurements_table_xct['id_sample'] == samples_table['id_sample'].iloc[i]]

        if measurements_table_registered_sample.empty:
            print(f"No xct measurements found for sample {samples_table['name_sample'].iloc[i]}")
            continue
        
        #check if the sample has already been registered
        if available_registrations:
            if measurements_table_reference_sample['id_measurement'].iloc[0] in registrations_table['reference_measurement_id_registration'].values:
                if measurements_table_registered_sample['id_measurement'].iloc[0] in registrations_table['registered_measurement_id_registration'].values:
                    print(f"Sample {samples_table['name_sample'].iloc[i]} has already been registered.")
                    continue

        #get the paths
        reference_measurement_path = Path(measurements_table_reference_sample['file_path_measurement'].iloc[0])
        registered_measurement_path = Path(measurements_table_registered_sample['file_path_measurement'].iloc[0])

        #load the reference measurement
        rf = io.load_tif(reference_measurement_path)

        #load the registered measurement
        xct = io.load_tif(registered_measurement_path)

        #create a folder named aux_registration in the same folder as the registered file
        aux_registration_folder = registered_measurement_path.parent / 'aux_registration'
        aux_registration_folder.mkdir(parents=True, exist_ok=True)

        #convert rf from z,y,x to x,y,z
        rf = np.transpose(rf, (2, 1, 0))

        #UT preprocessing
        amp = signal.envelope(rf)

        rf = np.transpose(amp, (2, 1, 0))  # Ensure rf is in the correct shape for registration
        amp = np.transpose(amp, (2, 1, 0))  # Ensure amp is in the correct shape for registration
        
        #register
        parameters,ut_centers,xct_centers, transformed = register.register_ut_xct_monoelement(amp,xct,reference_resolution,registered_resolution)

        # Save the aux files
        aux_output_file = aux_registration_folder / 'centers.tif'
        io.save_tif(aux_output_file, ut_centers)
        aux_output_file = aux_registration_folder / 'centers_xct.tif'
        io.save_tif(aux_output_file, xct_centers)
        aux_output_file = aux_registration_folder / 'transformed_centers_xct.tif'
        io.save_tif(aux_output_file, transformed)

        #estimate the registration error
        mse = np.mean(((ut_centers > 0)*1 - (transformed> 0)*1) ** 2)
        print(f"Mean Squared Error (MSE): {mse}")

        #save into the database
        axes = ['x', 'y']

        parameters_list = []

        for i in range(len(parameters)):
            aux_list = []
            for j in range(len(parameters[i])):
                aux_list.append(float(parameters[i][j]))
            parameters_list.append(aux_list)

        load.load_registration(conn,parameters_list,str(reference_measurement_path),str(registered_measurement_path),registration_type,axes)
    
    except Exception as e:
        print(f"Error processing sample {samples_table['name_sample'].iloc[i]}: {e}")
        continue

Processing sample 1/24: 1.7
Preprocessing
Preprocessed
Registering
Registered
Mean Squared Error (MSE): 0.004203323558162268
Registration loaded with ID: 55
Processing sample 2/24: 1.8
Preprocessing
Preprocessed
Registering
Registered
Mean Squared Error (MSE): 0.0022047546012269937
Registration loaded with ID: 56
Processing sample 3/24: 1.9
Preprocessing
Preprocessed
Registering
Registered
Mean Squared Error (MSE): 0.004777830864787387
Registration loaded with ID: 57
Processing sample 4/24: 1.10
Preprocessing
Preprocessed
Registering
Registered
Mean Squared Error (MSE): 0.0028640059127864005
Registration loaded with ID: 58
Processing sample 5/24: 1.11
Preprocessing
Preprocessed
Registering
Registered
Mean Squared Error (MSE): 0.00501573564122738
Registration loaded with ID: 59
Processing sample 6/24: 1.12
Preprocessing
Preprocessed
Registering
Registered
Mean Squared Error (MSE): 0.0
Registration loaded with ID: 60
Processing sample 7/24: 1.13
Preprocessing
Preprocessed
Registering
Reg